# setup compute

In [7]:
!pip install dask-jobqueue
!conda install -c conda-forge ipywidgets -y



Client 1: <Client: No scheduler connected>


In [10]:
#pip install prefect[dask] dask-jobqueue
# prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Simplify SLURMCluster parameters to basic working configuration
cluster_kwargs = {
    "cores": 12,  
    "memory": "10GB",  # Correct spacing
    "shebang": "#!/bin/bash",
    "account": "facet",
    "walltime": "00:10:00",
    "job_script_prologue": ["source ~/.bashrc"],
    # Only basic directives, add more as needed and test each step
    "job_extra_directives": ["-q debug", "--partition=milano", "--qos=preemptable"],
}




In [3]:
cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())


#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -A facet
#SBATCH -n 1
#SBATCH --cpus-per-task=12
#SBATCH --mem=10G
#SBATCH -t 00:10:00
#SBATCH -q debug
#SBATCH --partition=milano
#SBATCH --qos=preemptable
source ~/.bashrc
/sdf/home/s/sanjeev/miniforge3/envs/dask/bin/python3.12 -m distributed.cli.dask_worker tcp://172.24.49.15:46467 --name dummy-name --nthreads 3 --memory-limit 2.33GiB --nworkers 4 --nanny --death-timeout 60



/sdf/home/s/sanjeev/miniforge3/envs/dask/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46745 instead
  warnings.warn(


In [4]:
slurm_jobs = 1
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)
client

# Non Parallel Code 

In [ ]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [ ]:
%time costly_simulation([1, 2, 3, 4])


In [ ]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

In [ ]:
results = []


In [ ]:
%%time
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)

In [ ]:
results


# Parrellel Code

In [ ]:
import dask
lazy_results = []

In [ ]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

In [ ]:
lazy_results[0]


# shutdown clients

In [ ]:
from dask.distributed import Client

# Access the global list of clients
clients = Client._instances

# Display information about each client
for i, client in enumerate(clients):
    print(f"Client {i+1}: {client}")
client.shutdown()